This is currently under construction.

This notebook contains the functions that are called inside the rubix pipeline.

# Config

contains all the information needed to run the pipeline. Those are run specfic configurations.

In [ ]:
import os
config = {
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["gas"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 11,
        "reuse": True,
        }
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-11.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE"},
        
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1}

        
}

In [ ]:
# NBVAL_SKIP
from rubix.core.data import convert_to_rubix, prepare_input

convert_to_rubix(config) # Convert the config to rubix format and store in output_path folder
coords, vel, metallicity, mass, age = prepare_input(config) # Prepare the input for the pipeline

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
# Make a scatter plot 
n_subset = 10000
subset = coords[:n_subset]

plt.scatter(subset[:,0], subset[:,1], s=1)


Get the function that does the binning

In [ ]:
# NBVAL_SKIP
from rubix.core.telescope import get_spatial_bin_edges,get_spaxel_assignment

In [ ]:
# NBVAL_SKIP
bin_particles = get_spaxel_assignment(config)
edges = get_spatial_bin_edges(config)

In [ ]:
# NBVAL_SKIP
# Make a scatter plot of the spaxels
spaxel_index = bin_particles(coords)

print("Shape of spaxel_index", spaxel_index.shape)

In [ ]:
# NBVAL_SKIP
# Calculate binned image


spatial_bin_edges = edges
pixel_assignments = spaxel_index
import numpy as np
image = np.zeros((len(spatial_bin_edges) - 1, len(spatial_bin_edges) - 1))

# Count the number of particles in each pixel
for i in range(len(spatial_bin_edges) - 1):
    for j in range(len(spatial_bin_edges) - 1):
        image[i, j] = np.sum(pixel_assignments == (i + (len(spatial_bin_edges) - 1) * j))

image = np.log10(image + 1)
plt.imshow(image, cmap="viridis")